In [1]:
import numpy as np
from tifffile import imread, imwrite 
import sys
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import glob
sys.path.append("../NEAT")
from  NEATUtils import Augmentation
from NEATUtils import MovieCreator, npzfileGenerator
try:
    from pathlib import Path
    Path().expanduser()
except (ImportError,AttributeError):
    from pathlib2 import Path

try:
    import tempfile
    tempfile.TemporaryDirectory
except (ImportError,AttributeError):
    from backports import tempfile
import scipy.ndimage as ndi
import csv
from stardist.models import StarDist2D
from csbdeep.models import Config, CARE
from csbdeep.utils import normalize
from skimage.filters import threshold_otsu
import cv2
from skimage.measure import regionprops
from skimage import measure, morphology
from skimage.filters import sobel
from skimage.segmentation import watershed
import matplotlib.pyplot  as plt

Using TensorFlow backend.


In [2]:
#Normal : Label 0 Apoptosis : Label 1 Division : Label 2 Macrocheate : Label 3 Non MatureP1 : Label 4 Mature Event: Label 5

In [3]:
ModelDirectory = '/home/sancere/VarunNewton/CurieDeepLearningModels/O-NEATweights/'

DenoisingModel = 'BorialisS1S2FlorisMidNoiseModel'
SegmentationMaskModel = 'DrosophilaMaskSegmentationCARE'
SegmentationStarModel = 'DrosophilaSegmentationSmartSeeds'

model = StarDist2D(config = None, name = SegmentationStarModel, basedir = ModelDirectory)
UnetModel = CARE(config = None, name = SegmentationMaskModel, basedir = ModelDirectory)
NoiseModel = CARE(config = None, name = DenoisingModel, basedir = ModelDirectory)






Loading network weights from 'weights_best.h5'.






Couldn't load thresholds from 'thresholds.json', using default values. (Call 'optimize_thresholds' to change that.)
Using default values: prob_thresh=0.5, nms_thresh=0.4.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.


In [14]:

Dir = '/home/sancere/VarunNewton/CurieTrainingDatasets/Raw_Datasets/Neat/StaticMacroKittyEventTrainingDataBin2/ShiftLXY/'
cellX = 0.314814814814815
cellY = 0.314814814814815
trainlabel = 2

In [15]:
axis_norm = (0,1)
TotalCategories = 6
Label = np.zeros([TotalCategories + 4])
Label[trainlabel] = 1

Subdir = next(os.walk(Dir))
for x in Subdir[1]:
    currentdir = Dir  + x
    Path(currentdir + '/Segmentation/').mkdir(exist_ok = True)
    Raw_path = os.path.join(currentdir, '*tif')
    X = glob.glob(Raw_path)
    for fname in X:
        image = imread(fname)
        #Get Name
        Name = os.path.basename(os.path.splitext(fname)[0])



        x = image

        originalX = x.shape[0]
        originalY = x.shape[1]
       
        #Stardist Prediction

        scale_percent = 400 # percent of original size
        width = int(x.shape[1] * scale_percent / 100)
        height = int(x.shape[0] * scale_percent / 100)
        dim = (width, height)
        # resize image
        x = cv2.resize(x, dim)
        x = normalize(x,1,99.8,axis=axis_norm)
       

        MidImage, details = model.predict_instances(x)
       
        

        segimage = MidImage
       

        #Save different method segmentation

        imwrite((currentdir + '/Segmentation/'  + Name+ '.tif' ) , segimage.astype('float32'))
     
        smallimg = segimage.astype('uint16')
        centralLabel = smallimg[int(smallimg.shape[0]*cellY), int(smallimg.shape[1]*cellX)]
        
        
        if len(regionprops(smallimg)) > 0:
         for region in regionprops(smallimg):
           
            if region.label == centralLabel and centralLabel > 0:
              minr, minc, maxr, maxc = region.bbox
              
              center = region.centroid
              height = 0.25 * abs(maxc - minc)
              width = 0.25 * abs(maxr - minr)
            if centralLabel == 0:
            
               center = (smallimg.shape[0]*cellY,smallimg.shape[1]*cellX)
               height = 10
               width = 10 
               
        else:
               center = (smallimg.shape[0]*cellY,smallimg.shape[1]*cellX)
               height = 10
               width = 10 
            
              
        print(centralLabel, Name, 0.25 * center[0]/originalY, 0.25 * center[1]/originalX, height/originalY, width/originalX)
            
        Label[TotalCategories] = 0.25 * center[1]/originalX
        Label[TotalCategories + 1] = 0.25 * center[0]/originalY
       
        Label[TotalCategories + 2] = height/originalY
        Label[TotalCategories + 3] = width/originalX
        
        writer = csv.writer(open(currentdir + '/' + (Name) + 'DetectionLabel'  +".csv", "w"))
   
        for l in Label : writer.writerow ([l])
        
        

0 127Bin2 0.314814814814815 0.314814814814815 0.18518518518518517 0.18518518518518517
12 102Bin2 0.40454402467073036 0.22558711593799316 0.28703703703703703 0.35648148148148145
0 109Bin2 0.314814814814815 0.314814814814815 0.18518518518518517 0.18518518518518517
32 39Bin2 0.2837630822623502 0.23102733040507564 0.2037037037037037 0.2175925925925926
26 168Bin2 0.34813502587619943 0.18066145459233474 0.3333333333333333 0.3472222222222222
0 51Bin2 0.314814814814815 0.314814814814815 0.18518518518518517 0.18518518518518517
16 253Bin2 0.23591995558907325 0.27437363834422657 0.2962962962962963 0.3333333333333333
0 114Bin2 0.314814814814815 0.314814814814815 0.18518518518518517 0.18518518518518517
0 106Bin2 0.314814814814815 0.314814814814815 0.18518518518518517 0.18518518518518517
12 30Bin2 0.3436341161928307 0.23162168658151353 0.18981481481481483 0.25925925925925924
15 79Bin2 0.26640019379844965 0.322609819121447 0.22685185185185186 0.2638888888888889
7 144Bin2 0.32215327029156815 0.3039584

0 98Bin2 0.314814814814815 0.314814814814815 0.18518518518518517 0.18518518518518517
7 44Bin2 0.22036279152423663 0.22979322817030975 0.30092592592592593 0.35185185185185186
0 27Bin2 0.314814814814815 0.314814814814815 0.18518518518518517 0.18518518518518517
9 5Bin2 0.37832772873756476 0.2774703034265876 0.24537037037037038 0.2962962962962963
7 199Bin2 0.3048871063576946 0.3302510398098633 0.1388888888888889 0.11574074074074074
14 149Bin2 0.458461173204763 0.31613756613756616 0.3194444444444444 0.3287037037037037
14 156Bin2 0.32212735681252225 0.16694829835171351 0.3425925925925926 0.23148148148148148
0 108Bin2 0.314814814814815 0.314814814814815 0.18518518518518517 0.18518518518518517
15 86Bin2 0.2932225826125527 0.3284108865602129 0.2962962962962963 0.25462962962962965
6 162Bin2 0.3212198186386977 0.31232929094286027 0.0787037037037037 0.12962962962962962
10 145Bin2 0.37238649940262847 0.2949725752145107 0.2824074074074074 0.28703703703703703
5 43Bin2 0.3287795992714026 0.27571966855

10 42Bin2 0.2121208863662424 0.2728579004701296 0.25462962962962965 0.28703703703703703
32 64Bin2 0.25986669293406883 0.363672182821119 0.2175925925925926 0.25462962962962965
16 192Bin2 0.29242739060335626 0.2550468923859482 0.21296296296296297 0.2638888888888889
0 143Bin2 0.314814814814815 0.314814814814815 0.18518518518518517 0.18518518518518517
10 12Bin2 0.29398148148148145 0.27020487682252387 0.125 0.1111111111111111
11 82Bin2 0.20266029116960793 0.2906684412895593 0.28703703703703703 0.32407407407407407
18 237Bin2 0.3380089616208845 0.20997981470658267 0.28703703703703703 0.35185185185185186
0 219Bin2 0.314814814814815 0.314814814814815 0.18518518518518517 0.18518518518518517
0 139Bin2 0.314814814814815 0.314814814814815 0.18518518518518517 0.18518518518518517
19 96Bin2 0.2760388593721927 0.3741273324606658 0.16666666666666666 0.23148148148148148
2 232Bin2 0.3243135376756066 0.3392667092379736 0.07407407407407407 0.08333333333333333
14 25Bin2 0.36022219004408407 0.2916706889339382

0 252Bin2 0.314814814814815 0.314814814814815 0.18518518518518517 0.18518518518518517
11 57Bin2 0.22603974012424716 0.3758496546055231 0.2175925925925926 0.2824074074074074
0 183Bin2 0.314814814814815 0.314814814814815 0.18518518518518517 0.18518518518518517
3 226Bin2 0.2730052125100241 0.2614125753660637 0.24537037037037038 0.3333333333333333
5 131Bin2 0.30662704188905066 0.2866124858482937 0.09259259259259259 0.08796296296296297
10 141Bin2 0.2603081597222222 0.3004195601851852 0.05555555555555555 0.125
2 182Bin2 0.2851927130382415 0.3269911171333935 0.11574074074074074 0.14351851851851852
0 150Bin2 0.314814814814815 0.314814814814815 0.18518518518518517 0.18518518518518517
19 112Bin2 0.34825971506709913 0.30431798426932943 0.30092592592592593 0.3472222222222222
17 161Bin2 0.41227328151196463 0.29493725549966976 0.36574074074074076 0.27314814814814814
28 246Bin2 0.3026109148017275 0.2570180604632901 0.14351851851851852 0.12962962962962962
0 213Bin2 0.314814814814815 0.314814814814815 

6 214Bin2 0.3230452674897119 0.344483024691358 0.12037037037037036 0.0787037037037037
0 111Bin2 0.314814814814815 0.314814814814815 0.18518518518518517 0.18518518518518517
0 195Bin2 0.314814814814815 0.314814814814815 0.18518518518518517 0.18518518518518517
22 202Bin2 0.27723592783833745 0.27081341237967743 0.12962962962962962 0.14351851851851852
25 105Bin2 0.39723179605011216 0.3366260462826194 0.18055555555555555 0.19907407407407407
0 164Bin2 0.314814814814815 0.314814814814815 0.18518518518518517 0.18518518518518517
0 245Bin2 0.314814814814815 0.314814814814815 0.18518518518518517 0.18518518518518517
18 220Bin2 0.24445030826966896 0.22394506991913504 0.28703703703703703 0.33796296296296297
17 157Bin2 0.36442993458265704 0.20806895873296935 0.32407407407407407 0.35185185185185186
6 37Bin2 0.45657497092043614 0.358898890084684 0.2638888888888889 0.37962962962962965
7 99Bin2 0.32169274159216693 0.32662170072371227 0.14351851851851852 0.1574074074074074
9 62Bin2 0.23132360834833127 0.30

13 178Bin2 0.35847359906374776 0.2405987539584194 0.21296296296296297 0.2916666666666667
19 225Bin2 0.23499732203719934 0.24013819911059175 0.2361111111111111 0.24537037037037038
8 257Bin2 0.31136831275720167 0.3514789094650206 0.11574074074074074 0.1111111111111111
12 223Bin2 0.23383216351035183 0.32625369708206414 0.13425925925925927 0.19907407407407407
33 28Bin2 0.292614713689799 0.29586967513588674 0.09722222222222222 0.1111111111111111
0 174Bin2 0.314814814814815 0.314814814814815 0.18518518518518517 0.18518518518518517
0 169Bin2 0.314814814814815 0.314814814814815 0.18518518518518517 0.18518518518518517
10 171Bin2 0.31197526866244485 0.2516281663788141 0.33796296296296297 0.2777777777777778
0 190Bin2 0.314814814814815 0.314814814814815 0.18518518518518517 0.18518518518518517
7 191Bin2 0.2757240943515453 0.2369398938026389 0.3333333333333333 0.32407407407407407
16 18Bin2 0.3305160142348754 0.30749967048899435 0.1712962962962963 0.18055555555555555
13 7Bin2 0.2751143435143039 0.183